Imports

In [784]:
import cv2
import numpy as np
import os

In [785]:
show_steps = False

Functions

In [786]:
def edge_consistency_mask(gray):
    
    # Find edges
    edges = cv2.Canny(gray, 50, 150)
    
    # Dilate edges to create boundaries
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    edge_dilated = cv2.dilate(edges, kernel, iterations=1)
    
    # Invert 
    consistent_mask = cv2.bitwise_not(edge_dilated)
    
    return consistent_mask

In [787]:
def remove_noise_and_erode(noise_mask):

    # Removes noise
    noise_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    no_noise_mask = cv2.morphologyEx(noise_mask, cv2.MORPH_OPEN, noise_kernel, iterations=1)

    # Erode to separate shapes connected to random blotches
    erosion_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    eroded_mask = cv2.erode(no_noise_mask, erosion_kernel, iterations=2)

    return eroded_mask

In [788]:
def smooth_mask_and_fill_gaps(unsmoothed_mask):
    
    smoothing_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

    # Fill the gaps and smooth the edges
    filled_gaps_mask = cv2.morphologyEx(unsmoothed_mask, cv2.MORPH_CLOSE, smoothing_kernel)

    # Smooth more with blur
    smoothed_mask = cv2.medianBlur(filled_gaps_mask, 15)

    return smoothed_mask

In [789]:
def heavy_erode(non_eroded_mask):
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))

    # Remove small noise while keeping size of surviving shapes
    opened_mask = cv2.morphologyEx(non_eroded_mask, cv2.MORPH_OPEN, kernel, iterations=3)

    # Heavy erosion to separate shapes
    separated_mask = cv2.erode(opened_mask, kernel, iterations=7)

    return separated_mask

In [790]:
def filter_countours_compactness(old_mask, contours, min_area=500, max_compactness=55, min_perimeter=0):

    # Create empty mask from old mask shape
    new_mask = np.zeros_like(old_mask)

    for contour in contours:

        # Get area/perimeter for each contour
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        
        # Filter by area and compactness of shape
        if area > min_area and perimeter > min_perimeter and (perimeter * perimeter) / area < max_compactness:
            # Draw passing contours on new mask
            cv2.fillPoly(new_mask, [contour], 255)
    
    return new_mask

In [791]:
def dilate_to_undo_erosion(clean_mask):

    # Undo erosion that filled gaps
    erosion_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    final_mask = cv2.dilate(clean_mask, erosion_kernel, iterations=2)

    # Undo heavy erosion that separated shapes
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
    final_mask = cv2.dilate(final_mask, kernel, iterations=7)

    return final_mask

In [792]:
def get_rigid_contours(contours, epsilon_size=0.01):

    rigid_contours = []

    for contour in contours:
        
        # Approximate contour with fewer points
        epsilon = epsilon_size * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        rigid_contours.append(approx)
    
    return rigid_contours

In [793]:
def get_hull_contours(contours):

    hull_contours = []
    
    for contour in contours:
        
        # Get convex hull of contour
        hull = cv2.convexHull(contour)
        hull_contours.append(hull)
    
    return hull_contours

In [794]:
def filter_noise_contours(contours, min_area=1000):

    # Remove noise
    filtered_contours = []

    for contour in contours:
        
        area = cv2.contourArea(contour)

        # How much noise to filter
        if area > min_area:
            filtered_contours.append(contour)
    
    return filtered_contours

In [795]:
def get_centers(contours):

    # Calculate centers of contours
    centers = []

    for contour in contours:

        # Calculate moments
        M = cv2.moments(contour)
        
        if M["m00"] != 0: 

            # x coord
            cx = int(M["m10"] / M["m00"])

            # y coord
            cy = int(M["m01"] / M["m00"])

            centers.append((cx, cy))
    
    return centers

File Names/Dirs

In [796]:
file_dir = "data"
file_name = "PennAir 2024 App Static.png"
file = os.path.join(file_dir, file_name)

In [797]:
output_dir = os.path.join("output", "consistency")
output_name = f"annotated_{file_name}"
output = os.path.join(output_dir, output_name)

Load Image

In [798]:
img = cv2.imread(file)

In [799]:
img = cv2.resize(img, (1920, 1080))

Process Image

In [800]:
# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Consistency detection to create mask
consistency_mask = edge_consistency_mask(gray)

# Remove noise and erode
cleaned_mask = remove_noise_and_erode(consistency_mask)

# Smooth
smooth_mask = smooth_mask_and_fill_gaps(cleaned_mask)

# Erode
eroded_mask = heavy_erode(smooth_mask)

# Find contours
contours, _ = cv2.findContours(eroded_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours by compactness
clean_mask = filter_countours_compactness(eroded_mask, contours)

# Undo erosion
final_mask = dilate_to_undo_erosion(clean_mask)

# Find/Draw Outlines and Centers
contours, _ = cv2.findContours(final_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Make contours more rigid
rigid_contours = get_rigid_contours(contours)

# Make contours convex
hull_contours = get_hull_contours(rigid_contours)

filtered_contours = filter_noise_contours(hull_contours)

centers = get_centers(filtered_contours)

# Copy original
result = img.copy()

# Draw outlines of shapes
cv2.drawContours(result, filtered_contours, -1, (0, 255, 0), 2)

# Draw centers and place text
for i, (contour, center) in enumerate(zip(filtered_contours, centers)):
        cx, cy = center
        
        # Get bounding box for text placement
        x, y, w, h = cv2.boundingRect(contour)
        
        # Draw center
        cv2.circle(result, center, 5, (255, 255, 255), -1)
        
        # Place text
        text = f"({cx},{cy})"
        text_x = cx - 80
        text_y = y + h + 40
        cv2.putText(result, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)

if show_steps:
    # Display results
    cv2.imshow("Image Results", result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

Save Result

In [801]:
cv2.imwrite(output, result)

True